<a href="https://colab.research.google.com/github/Aleks-cmyk/neural-netw/blob/master/lab5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Лабораторна робота 5

студентки КН-31

Мензатюк Олександри

# Завдання
На основі документу вирішити завдання класифікації зображень їжі для 3 класів з набору даних food101.


Індекси класів визначити індивідуально за залежностями: i1 = n - 1 = 4, i2 = n + 29 = 34, i3 = n + 59 = 64 (де і1, і2, і3 - індекс класу (починаючи з 0) у відсортованому за алфавітом наборі даних, n = 5 - номер за списком групи.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random
import os
import zipfile
import pathlib
import numpy as np
import shutil
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense

In [ ]:
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/101_food_classes_10_percent.zip

zip_ref = zipfile.ZipFile("101_food_classes_10_percent.zip", "r")
zip_ref.extractall()
zip_ref.close()

--2022-05-21 07:12:51--  https://storage.googleapis.com/ztm_tf_course/food_vision/101_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.197.128, 74.125.135.128, 74.125.195.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.197.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1625420029 (1.5G) [application/zip]
Saving to: ‘101_food_classes_10_percent.zip.3’

101_food_classes_10 100%[===================>]   1.51G   239MB/s    in 7.6s    

2022-05-21 07:12:59 (203 MB/s) - ‘101_food_classes_10_percent.zip.3’ saved [1625420029/1625420029]



In [ ]:
!ls 101_food_classes_10_percent/train

apple_pie	    eggs_benedict	     onion_rings
baby_back_ribs	    escargots		     oysters
baklava		    falafel		     pad_thai
beef_carpaccio	    filet_mignon	     paella
beef_tartare	    fish_and_chips	     pancakes
beet_salad	    foie_gras		     panna_cotta
beignets	    french_fries	     peking_duck
bibimbap	    french_onion_soup	     pho
bread_pudding	    french_toast	     pizza
breakfast_burrito   fried_calamari	     pork_chop
bruschetta	    fried_rice		     poutine
caesar_salad	    frozen_yogurt	     prime_rib
cannoli		    garlic_bread	     pulled_pork_sandwich
caprese_salad	    gnocchi		     ramen
carrot_cake	    greek_salad		     ravioli
ceviche		    grilled_cheese_sandwich  red_velvet_cake
cheesecake	    grilled_salmon	     risotto
cheese_plate	    guacamole		     samosa
chicken_curry	    gyoza		     sashimi
chicken_quesadilla  hamburger		     scallops
chicken_wings	    hot_and_sour_soup	     seaweed_salad
chocolate_cake	    hot_dog		     shrimp_and_grits
chocolate_mousse    huevo

In [ ]:
train_dir = "101_food_classes_10_percent/train/"
test_dir = "101_food_classes_10_percent/test/"

In [ ]:
data_dir = pathlib.Path(train_dir)
all_class_names = np.array(sorted([item.name for item in data_dir.glob('*')]))

indexes = [4, 34, 64]

class_names = np.array([])

for i, name in enumerate(all_class_names):
  if i in indexes:
    print(i, name)
    class_names = np.append(class_names, name)

4 beef_tartare
34 eggs_benedict
64 miso_soup


In [ ]:
!mkdir beeftartare_eggsbenedict_misosoup

!mkdir beeftartare_eggsbenedict_misosoup/train
!mkdir beeftartare_eggsbenedict_misosoup/test

In [ ]:
sources = [
  'beef_tartare',
  'eggs_benedict',
  'miso_soup'
]

for source in sources:
  source_dir = train_dir + source
  destination_dir = 'beeftartare_eggsbenedict_misosoup/train/' + source
  shutil.copytree(source_dir, destination_dir)

for source in sources:
  source_dir = test_dir + source
  destination_dir = 'beeftartare_eggsbenedict_misosoup/test/' + source
  shutil.copytree(source_dir, destination_dir)

In [ ]:
train_dir = "beeftartare_eggsbenedict_misosoup/train/"
test_dir = "beeftartare_eggsbenedict_misosoup/test/"

In [ ]:
data_dir = pathlib.Path(train_dir)
class_names = np.array(sorted([item.name for item in data_dir.glob('*')]))
print(class_names)

['beef_tartare' 'eggs_benedict' 'miso_soup']


In [ ]:
train_datagen = ImageDataGenerator(rescale=1/255.)
test_datagen = ImageDataGenerator(rescale=1/255.)

train_data = train_datagen.flow_from_directory(test_dir,
                                               target_size=(224, 224),
                                               batch_size=32,
                                               class_mode='categorical')

test_data = train_datagen.flow_from_directory(train_dir,
                                              target_size=(224, 224),
                                              batch_size=32,
                                              class_mode='categorical')

Found 750 images belonging to 3 classes.
Found 225 images belonging to 3 classes.


In [ ]:
train_datagen_augmented = ImageDataGenerator(rescale=1/255.,
                                             rotation_range=20, 
                                             width_shift_range=0.2,
                                             height_shift_range=0.2,
                                             zoom_range=0.2,
                                             horizontal_flip=True)

train_data_augmented = train_datagen_augmented.flow_from_directory(test_dir,
                                                                  target_size=(224, 224),
                                                                  batch_size=32,
                                                                  class_mode='categorical')

Found 750 images belonging to 3 classes.


In [ ]:

cnn_model = Sequential([
  Conv2D(32, 3, activation='relu', input_shape=(224, 224, 3)),
  MaxPool2D(),
  Dense(128, activation='relu'),
  Conv2D(64, 3, activation='relu'),
  MaxPool2D(),
  Flatten(),
  Dense(3, activation='softmax')
])

cnn_model.compile(loss="categorical_crossentropy",
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])

result_cnn_model = cnn_model.fit(train_data_augmented,
                          epochs=50,
                          steps_per_epoch=len(train_data_augmented),
                          validation_data=test_data,
                          validation_steps=len(test_data))

Epoch 1/50
24/24 [==============================] - 13s 513ms/step - loss: 1.5333 - accuracy: 0.3947 - val_loss: 0.9649 - val_accuracy: 0.4844
Epoch 2/50
24/24 [==============================] - 12s 483ms/step - loss: 0.9743 - accuracy: 0.4947 - val_loss: 0.8498 - val_accuracy: 0.6222
Epoch 3/50
24/24 [==============================] - 11s 477ms/step - loss: 0.9604 - accuracy: 0.4827 - val_loss: 0.8814 - val_accuracy: 0.6133
Epoch 4/50
24/24 [==============================] - 11s 476ms/step - loss: 0.9422 - accuracy: 0.5480 - val_loss: 0.8143 - val_accuracy: 0.6044
Epoch 5/50
24/24 [==============================] - 12s 480ms/step - loss: 0.8466 - accuracy: 0.6107 - val_loss: 0.7231 - val_accuracy: 0.6933
Epoch 6/50
24/24 [==============================] - 11s 477ms/step - loss: 0.7969 - accuracy: 0.6413 - val_loss: 0.8312 - val_accuracy: 0.6533
Epoch 7/50
24/24 [==============================] - 12s 482ms/step - loss: 0.8491 - accuracy: 0.6133 - val_loss: 0.7512 - val_accuracy: 0.6756

# Висновок
В ході виконання лабораторної роботи було створено нейромережу для розв'язання задачі класифікації зображень їжі. Спочатку було розділено зображення у відношенні 1:3 - (тренувальна вибірка:тестова вибірка). У такому варіанті найкращою точністю передбачень було значення 0.65. після цього було розділено всі зображення у відношенні 3:1, тобто тепер стало набагато більше навчальних прикладів. Після цього, точність передбачення зросла до 7.1